# Testing

Tests worden geschreven in de file `tests.py`.

We gaan nu een test schrijven voor "Questions" die in de toekomst worden gecreeerd. Maar dus niet als "recently_published" mogen worden aanzien.



In [ ]:
from django.test import TestCase

import datetime
from django.utils import timezone

from .models import Question


# Create your tests here.

class QuestionModelTests(TestCase):
    
    def test_was_published_recently_with_future_question(self):
        """
        was_published_recently() returns False for questions whose pub_date
        is in the future.
        """
        future_time = timezone.now() + datetime.timedelta(days=30)
        future_question = Question(pub_date=future_time)

        self.assertIs(future_question.was_published_recently(), False)

Deze test kunnen we nu runnen door in de terminal `python manage.py test polls` te schrijven.

Hierdoor gaan we dus enkel de tests van polls runnen.

met `python manage.py test` runnnen we alle tests van alle apps.

De uitkomst van de test is:

In [ ]:
Creating test database for alias 'default'...
System check identified no issues (0 silenced).
F
======================================================================
FAIL: test_was_published_recently_with_future_question (polls.tests.QuestionModelTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/path/to/mysite/polls/tests.py", line 16, in test_was_published_recently_with_future_question
    self.assertIs(future_question.was_published_recently(), False)
AssertionError: True is not False

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)
Destroying test database for alias 'default'...

We hadden "False" verwacht, maar hebben True gekregen.

Tijdens het uitvoeren van `python manage.py test polls` zoekt django naar tests in de "polls" applicatie.

In 'polls' vond het dus een "TestCase" object.

Tijdens het uitvoeren van de test wordt er hiervoor specifiek een database gemaakt.

Daarna wordt er gezocht naar test-methods. Dat zijn degenen die met "test" beginnen.

In het geval van onze test `test_was_published_recently_with_future_question()` werd er een "Question"-instance aangemaakt met 30 dagen in de toekomst.

Ee werd in deze methode de "assertIs()" methode gebruikt. We hadden False verwacht en er werd True gegenereerd.

## Fixen van de Bug

We zorgen dat onze methode `test_was_published_recently_with_future_question()` nu False geeft in het geval de datum in de toekomst ligt.


Wie weet hoe ?

In [ ]:
def was_published_recently(self):
    now = timezone.now()
    return now - datetime.timedelta(days=1) <= self.pub_date <= now

Als we nu de test met `python manage.py test polls` draaien, dan zullen we 'OK' krijgen.

# Meer Uitgebreide tests

We kunnen meer tests voor `was_published_recently` toevoegen.

We willen bv. checken of:
* Een test die ouder is dan 1 dag, als False geeft.
* Een test die jonger is als 1 dag, true geeft.


Hoe kunnen we dit doen ?

We voegen volgende toe aan onze "TestCase()":

In [ ]:
def test_was_published_recently_with_old_question(self):
    time = timezone.now() - datetime.timedelta(hours=24, seconds=1)
    old_question = Question(pub_date=time)
    
    self.assertIs(old_question.was_published_recently(), False)

def test_was_published_recently_with_recent_question(self):
    time = timezone.now() - datetime.timedelta(hours=23, minutes=59, seconds=59)
    recent_question = Question(pub_date=time)
    
    self.assertIs(recent_question.was_published_recently(), True)

# Test a view

Django heeft een (test) Client() die de mogelijkheid geeft om een "User" na te bootsen die met een "view interageert".

Vooraleer we onze test maken, gaan we eerst onze code aanmaken.


We beginnen met het updaten van onze `IndexView()`.

Ze ziet er nu nog zo uit:

In [ ]:
class IndexView(generic.ListView):
    template_name = "polls/index.html"
    context_object_name = "latest_question_list"

    def get_queryset(self) -> QuerySet[Any]:
        return Question.objects.order_by("-pub_date")[:3]

Deze view toont op dit moment ook "Question()"s die nog niet gepublished zijn. (die met een pub_date in de toekomst)

We zullen daarom onze queryset updaten om hier aan te voldoen.  (we voegen filter toe)

In [ ]:
return Question.objects.filter(pub_date__lte=timezone.now()).order_by("-pub_date")[:3]

in de filter gebruiken we `pub_date__lte`.

Dit staat voor "less than or equal". Dus we filteren enkel "Questions" waarvan de publicatie datum kleiner of gelijk zijn aan nu. 

Je heb nog andere "field lookups" zoals bv. __gte, __lt, __in, __contains, ... 

je vind een overzicht via: `https://docs.djangoproject.com/en/4.2/ref/models/querysets/#field-lookups`

## Testing our view

we updaten onze `test.py`. 

Eerst maken we een functie waarmee we makkelijk vragen kunnen aanmaken. Om herhaling te vermijden.

In [ ]:
def create_question(question_text, days):
    time = timezone.now() + datetime.timedelta(days=days)
    return Question.objects.create(question_text=question_text, pub_date=time)

Nu maken we een nieuwe "TestCase" aan voor onze view.

De client is een onderdeel van het "TestCase" object. We kunnen de client hierin oproepen via `self.client`.

In [ ]:
from django.urls import reverse


class QuestionIndexViewTests(TestCase):
    
    def test_no_questions(self):
        '''
        No question exists -> show message
        '''
        response = self.client.get(reverse("polls:index"))
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "No polls are available.")
        self.assertQuerysetEqual(response.context['latest_question_list'], [])

Als we nu onze tests runnen zien we dat er 4 test geslaagd zijn.

Maar wat hebben we nu eigenlijk gedaan. We laden de url, net zoals een user zou doen. En dan checken we of de response van deze `GET` voldoet aan een aantal voorwaarden. 

laten we even checken wat er gebeurd als we op onze  `index.html` het volgende veranderen (uitcommenten of verwijderen van `<p>No polls are available.</p>`)

In [ ]:
...

{% else %}
    {% comment %} <p>No polls are available.</p> {% endcomment %}
{% endif %}

Dan zien we dat deze test failed.

We zien dus dat de test daadwerkelijk gaat kijken wat er op de pagina staat.

We voegen nog een aantal tests toe. (vergeet niet om in `index.html` de tekst terug te zetten)

In [ ]:
def test_past_question(self):
    '''
    pub_date in past -> display
    '''
    question = create_question(question_text='Past question.', days=-30)
    response = self.client.get(reverse("polls:index"))
    self.assertQuerysetEqual(response.context['latest_question_list'], [question])

We maken specifiek voor onze test een Question aan, om onze "test-database" mee te vullen.



In [ ]:
def test_future_question(self):
    '''
    pub_date in future -> do not display
    '''
    create_question(question_text='Past question.', days=30)
    response = self.client.get(reverse("polls:index"))
    self.assertContains(response, 'No polls are available.')
    self.assertQuerysetEqual(response.context['latest_question_list'], [])

def test_future_and_past_questions(self):
    '''
    if both a future and past question exist -> only past is displayed
    '''
    question = create_question(question_text='Past question.', days=-30)
    create_question(question_text='Past question.', days=30)
    response = self.client.get(reverse("polls:index"))
    self.assertQuerysetEqual(response.context['latest_question_list'], [question])

def test_2_past_questions(self):
    question_1 = create_question(question_text='Past question 1.', days=-30)
    question_2 = create_question(question_text='Past question 2.', days=-5)
    response = self.client.get(reverse("polls:index"))
    self.assertQuerysetEqual(response.context['latest_question_list'], [question_2, question_1])

De database wordt bij elke test opniew "gereset".

## Testing the "DetailView"

Users kunnen nu misschien niet meer de links zien naar de verschillende Questions, maar ze kunnen nog altijd de details zien als ze de juiste url ingeven.

We moeten dus zorgen dat dit niet kan. Daarom voegen we ook bij de detailview een queryset bij.

In [ ]:
class DetailView(generic.DetailView):
    model = Question
    template_name = "polls/detail.html"

    def get_queryset(self) -> QuerySet[Any]:
        return Question.objects.filter(pub_date__lte=timezone.now())

We voegen nu tests toe om te checken of deze methode er inderdaad voor zorgt dat we de detail-pagina van de Questions met een "future date" niet kunnen bereiken. Maar wel een past question.

In [ ]:
class QuestionDetailViewTests(TestCase):

    def test_future_question(self):
        '''
        Detail future question -> 404 error
        '''
        future_question = create_question(question_text="Future question", days=5)
        url = reverse('polls:detail', args=(future_question.id,))
        response = self.client.get(url)
        self.assertEqual(response.status_code, 404)
    
    def test_past_question(self):
        '''
        Detail past question 
        '''
        past_question = create_question(question_text="Past Question",  days=-5)
        url = reverse('polls:detail', args=(past_question.id,))
        response = self.client.get(url)
        self.assertContains(response, past_question.question_text)

in de `test_future_question` doen we het volgende:

* we maken Question in de toekomst
* we maken de url via "reverse" (al het geen dat na het 'domain' komt) + halen de url op via "get"
* we checken of de status_code 404 is


Met de Question uit het verleden checken we of de text van de vraag in de "response" zit.

## Nog ideeën om te testen

Nog een aantal ideën die we zelf kunnen proberen als oefening:

* We zouden de `get_queryset` van de 'ResultsView' ook kunnen aanpassen zoals bij de DetailView.
* We zouden ook kunnen voorzien dat "Questions" enkel kunnen getoont worden wanneer ze een "Choice" hebben.
* We willen dat admins wel de 'unpublished questions' kunnen zien. Maar gewone gebruikers niet.

## Meer is beter

Het is over het algemeen altijd beter om meer tests te hebben.

éénmaal geschreven zal je een test normaal niet meer moeten aanpassen.

gebruik een TestCase per model / view